In [1]:
#from langchain.embeddings import HuggingFaceEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS

#from langchain.schema import Document
from langchain_core.documents import Document  # ✅ 최신 위치
import json

In [2]:

with open("langchain_docs.json", "r", encoding = "utf-8") as f:
    loaded_docs = json.load(f)

In [3]:

# ✅ loaded_docs 리스트에 dict 데이터가 여러개 있을 때 Document 객체로 변환
docs = []
for doc_data in loaded_docs:
    doc = Document(
        page_content=doc_data["content"],
        metadata={
            "title": doc_data["title"],
            "type": doc_data["type"]
        }
    )
    docs.append(doc)

In [4]:
docs

[Document(metadata={'title': 'LangChain overview', 'type': 'text'}, page_content='LangChain v1.0 is now available! For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide. If you encounter any issues or have feedback, please open an issue so we can improve. To view v0.x documentation, go to the archived content.LangChain is the easiest way to start building agents and applications powered by LLMs. With under 10 lines of code, you can connect to OpenAI, Anthropic, Google, and more. LangChain provides a pre-built agent architecture and model integrations to help you get started quickly and seamlessly incorporate LLMs into your agents and applications.We recommend you use LangChain if you want to quickly build agents and autonomous applications. Use LangGraph, our low-level agent orchestration framework and runtime, when you have more advanced needs that require a combination of deterministic and agentic workflows, heavy custo

In [5]:
# ✅ 임베딩 모델 설정 (GPU 서버에서 실행 가능)
#embedder = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")
#embedder = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")

embedder = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")


In [6]:
# ✅ FAISS 벡터 DB 생성
db = FAISS.from_documents(docs, embedding=embedder)

# ✅ 저장 (선택)
db.save_local("faiss_langchain_db")


In [7]:
query = "What is LangChain?"
retrieved_docs = db.similarity_search(query, k=3)

for i, doc in enumerate(retrieved_docs):
    print(f"--- Retrieved Document {i+1} ---")
    print(f"Title: {doc.metadata['title']}")
    print(f"Type: {doc.metadata['type']}")
    print(f"Content: {doc.page_content[:200]}...")  # 앞부분 200자만 출력
    print()

--- Retrieved Document 1 ---
Title: LangChain overview
Type: text
Content: LangChain v1.0 is now available! For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide. If you encounter any issues or have feedback, p...

--- Retrieved Document 2 ---
Title: Install
Type: code
Content: pip install -U langchain...

--- Retrieved Document 3 ---
Title: What's new in v1
Type: text
Content: LangChain v1 is a focused, production-ready foundation for building agents. We’ve streamlined the framework around three core improvements:...



In [9]:
query = "What is LangChain?"
retrieved_docs = db.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(retrieved_docs, 1):
    print(f"--- Retrieved Document {i} ---")
    print(f"Title: {doc.metadata['title']}")
    print(f"Type: {doc.metadata['type']}")
    print(f"Content: {doc.page_content[:200]}...")  # 앞부분 200자만 출력
    print(f"Score: {score}")
    print()

--- Retrieved Document 1 ---
Title: LangChain overview
Type: text
Content: LangChain v1.0 is now available! For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide. If you encounter any issues or have feedback, p...
Score: 0.22953097522258759

--- Retrieved Document 2 ---
Title: Install
Type: code
Content: pip install -U langchain...
Score: 0.23582054674625397

--- Retrieved Document 3 ---
Title: What's new in v1
Type: text
Content: LangChain v1 is a focused, production-ready foundation for building agents. We’ve streamlined the framework around three core improvements:...
Score: 0.23655475676059723

